#  *This is a very simple estimate of COVID-19 impacts, and I don't claim to know anything other than how to produce a nice figure and do simple math*

In [1]:
using Dates
using PlotlyJS

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-17772316287167429344\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-3920177953639290476\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-17772316287167429344\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
function cases(days::Day, current_cases, doubling_time)
    dbl_period = days/doubling_time
    return current_cases * (2 ^ dbl_period)
end

cases (generic function with 1 method)

In [18]:
function plot_covid(days, current_cases, doubling_time = Day(7), hosp_rate = 0.1, death_rate = 0.03)
    td = today()
    dates = td:doubling_time:(td+days)
    case_count = [cases(d-td, current_cases, doubling_time) for d in dates]
    total_cases = scatter(x = dates, y = case_count, name = "Infections")
    hosp = scatter(x = dates, y = case_count.*hosp_rate, name = "Hospitalizations")
    deaths = scatter(x = dates, y = case_count.*death_rate, name = "Deaths")
    plot([total_cases, hosp, deaths])
end

plot_covid (generic function with 4 methods)

## COVID-19 parameters
 - According to [this article](https://wwwnc.cdc.gov/eid/article/26/5/20-0146_article), the estimated doubling time of COVID-19 is ~7 days.
 - According to [the WHO](https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200306-sitrep-46-covid-19.pdf?sfvrsn=96b04adf_2), the mortality rate of COVID-19 is ~3%.
 - I don't know where the 10% hospitalization rate comes from.


## Colorado
According to [CDPHE](https://docs.google.com/document/d/e/2PACX-1vRSxDeeJEaDxir0cCd9Sfji8ZPKzNaCPZnvRCbG63Oa1ztz4B4r7xG_wsoC9ucd_ei3--Pz7UD50yQD/pub) there are currently 33 positive cases in Colorado

In [20]:
co = plot_covid(Day(60), 33)

data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with field margin"

In [21]:
savefig(co, "CO.html")

## US
According to the [CDC](https://www.cdc.gov/coronavirus/2019-ncov/cases-in-us.html) there are currently 1219 cases in the US.

In [25]:
us = plot_covid(Day(100), 1219)

data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with field margin"

In [24]:
savefig(us, "US.html")